# Create a basic optimization model

In this example, we explain the basic functions of the linopy Model class. First, we are setting up a very simple linear optimization model, given my 

     Minimize:
$$ x + 2y $$
      
     subject to:
$$ x \ge 0 $$
$$ y \ge 0 $$
$$   3x + 7y \ge 10 $$
$$ 5x + 2y \ge 3 $$




In [27]:
from linopy import Model

In [28]:
m = Model()

The model `m` serves at a container for all relevant data. 

Let's add the two variables. Note that a variable can always be assigned with a lower and an upper bound. In this case, both `x` and `y` have a lower bound of zero. Note, the default for lower and upper bounds are minus and plus infinity.

In [29]:
m.add_variables('x', lower=0)
m.add_variables('y', lower=0);

The first step for assigning the constraints, is to define the linear expressions on the left hand side (lhs) of the constraints. For this purpose,we use the 

Have a look at the lhs of the third variable. Each term consists of one coefficient and a variable, e.g. `+3x`. To define such a term, we use a tuple of the form (coefficient, variable). These tuples are then combined in a list (or in a tuple), which contains all the terms. 

When assigning to the model 

In [15]:
lhs = (3, 'x'), (7, 'y')
m.add_constraints('Constraint1', lhs, '>=', 10)

lhs = (5, 'x'), (2, 'y')
m.add_constraints('Constraint2', lhs, '>=', 3)

obj = (2, 'y'), (1, 'x')
m.add_objective(obj)

Linear Expression with 2 terms: 

Coefficients:
-------------
 <xarray.DataArray (term_: 2)>
array([2., 1.])
Coordinates:
  * term_    (term_) object 'y' 'x'

Variables:
----------
<xarray.DataArray 'y' (term_: 2)>
array([2., 1.])
Coordinates:
  * term_    (term_) object 'y' 'x'

In [16]:
m.solve()

Academic license - for non-commercial use only - expires 2021-04-18
Using license file /opt/gurobi911/gurobi.lic
Read LP format model from file /tmp/linopy-problem-ijvtkl2g.lp
Reading time = 0.00 seconds
obj: 2 rows, 2 columns, 4 nonzeros
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x2173ee9f
Coefficient statistics:
  Matrix range     [2e+00, 7e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 1e+01]
Presolve removed 2 rows and 2 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8620690e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  2.862068966e+00


<Linopy model>
Variables: x, y
Constraints: Constraint1, Constraint2
Dimensions: 
Status: optimal

In [17]:
m.solution

<xarray.Dataset>
Dimensions:  ()
Data variables:
    x        float64 0.03448
    y        float64 1.414

## How it works under the hook

The model stores all variables and constraints in `xr.Dataset`s. Variables are stored in form of `xarray.DataArray`s in `m.variables` which itself is a `xarray.Dataset`  . 

In [32]:
m.variables

<xarray.Dataset>
Dimensions:  ()
Data variables:
    x        int64 1
    y        int64 2